### 4-1. 인구현황 데이터 분석

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

def download_one_episode(title, no, url):
    # 기본 설정
    req_header = {'referer': url}
    imgdir_name = f'img/{title}/{no}'

    # 이미지 저장 폴더가 없으면 생성
    os.makedirs(imgdir_name, exist_ok=True)

    # 웹 페이지 요청 및 확인
    res = requests.get(url)
    if not res.ok:
        print(f'Error Code = {res.status_code}')
        exit()

    # 이미지 URL 추출
    soup = BeautifulSoup(res.text, 'html.parser')
    img_url_list = [img_tag['src'] for img_tag in soup.select("img[src*='IMAG01']")]

    # 이미지 다운로드
    for img_url in img_url_list:
        res = requests.get(img_url, headers=req_header)
        if res.ok:
            img_data = res.content
            file_path = os.path.join(imgdir_name, os.path.basename(img_url))
            with open(file_path, 'wb') as file:
                print(f'Writing to {file_path} ({len(img_data):,} bytes)')
                file.write(img_data)
        else:
            print(f'Error Code = {res.status_code} for {img_url}')

download_one_episode('입학용병',271,'https://comic.naver.com/webtoon/detail?titleId=758150&no=272&week=sun')

200


In [ ]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from urllib.parse import urlparse, parse_qs

def download_all_episode(title,episode_url):
    # url을 파싱해서 titleId를 알아내기
    parsed_url = urlparse(episode_url)
    query_params = parse_qs(parsed_url.query)
    title_id = query_params.get('titleId', [''])[0]
    
    api_url = f'https://comic.naver.com/api/article/list?titleId={title_id}'
    res = requests.get(api_url)
    print(res.status_code)    
    if res.ok:
        #pprint(res.json()['articleList'])
        for article in res.json()['articleList']:
            no = article['no']
            detail_url = f'https://comic.naver.com/webtoon/detail?titleId={title_id}&no={no}'
            print(detail_url)
        

if __name__ == '__main__': 
    download_all_episode('삽가능','https://comic.naver.com/webtoon/list?titleId=838708')